In [1]:
# Importing Libraries
import numpy as np
from qiskit import *
from qiskit.utils import QuantumInstance
from qiskit.extensions import UnitaryGate
from math import sqrt, ceil

In [2]:
# Implement Diffuser 
def diffuser(n):
    qc=QuantumCircuit(n)
    
    # Apply transformation |s> -> |00..0> (H-gates)
    qc.h(range(n))

    # Apply transformation |00..0> -> |11..1> (X-gates)
    qc.x(range(n))

    # Do multi-controlled-Z gate
    qc.h(n-1)
    qc.mct(list(range(n-1)), n-1)  # multi-controlled-toffoli
    qc.h(n-1)

    # Apply transformation |11..1> -> |00..0>
    qc.x(range(n))

    # Apply transformation |00..0> -> |s>
    qc.h(range(n))
    
    print("Diffuser Circuit: ")
    print(qc)
    gate = qc.to_gate()
    gate.name = "diffuser"
    return gate

In [3]:
# Define Oracle for the element to search
def oracle(n,e):
    matrix=np.identity(2**n)
    matrix[e][e]=-1
    #print(matrix)
    return UnitaryGate(matrix)
    

In [4]:
# Implementation of Grover Algothm 
def grover(n, l, r_i):
    grover_circuit = QuantumCircuit(n)
    
    # Generate the Oracle
    grover_circuit.append(oracle(n,l), range(n))
    
    # Attach the Diffuser for r_i times
    for i in range(r_i):
        grover_circuit.append(diffuser(n), range(n))
    
    print("Grover Circuit: ")
    print(grover_circuit)
    gate = grover_circuit.to_gate()
    gate.name = "Grover"
    
    return gate

In [5]:
def missingElement(X: list):
    
    # Claculte the number of qubits and rotations required
    n=ceil(np.log2(len(X)))
    r_i=ceil(sqrt(len(X)))+1
        
    qc = QuantumCircuit(n, n)
    for i in range(2**n):
        if i not in X:
            oracle(n,i)
            break
    
    # Convert Each State to Superposition state
    qc.h(range(n))
    qc.barrier()
    
    # Add the Grover Circuit
    qc.append(grover(n, i, r_i), range(n))
    qc.barrier()

    # Add Measurements to the circuit
    qc.measure(range(n), range(n))
    
    print("Missing Element Circuit")
    print(qc)

    # Execute circuit on qasm_simulator and extract the result
    backend = Aer.get_backend('qasm_simulator')
    instance = QuantumInstance(backend, shots=100000)
    result = instance.execute(qc)
    counts = result.get_counts()
    m=max(counts, key=counts.get)
    return int(m,2)

    

In [6]:
# Creating Random numbers of length (2^n)-1 with each value of 2^n except one value
import random
n=5
X=random.sample(range(2**n), 2**n-1)
print("Elements: ")
print(X)
ans = missingElement(X)
print("Missing Element: ",ans)

Elements: 
[29, 13, 23, 11, 10, 22, 2, 31, 9, 15, 1, 7, 18, 12, 24, 27, 26, 16, 17, 19, 14, 3, 4, 0, 5, 30, 21, 6, 28, 8, 25]
Diffuser Circuit: 
     ┌───┐┌───┐          ┌───┐┌───┐     
q_0: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤       │  ├───┤├───┤     
q_1: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤       │  ├───┤├───┤     
q_2: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤       │  ├───┤├───┤     
q_3: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐
q_4: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘
Diffuser Circuit: 
     ┌───┐┌───┐          ┌───┐┌───┐     
q_0: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤       │  ├───┤├───┤     
q_1: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤       │  ├───┤├───┤     
q_2: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤       │  ├───┤├───┤     
q_3: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
     ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐
q_4: ┤ H ├┤ X ├┤